In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tqdm

In [2]:
def cm_to_iou(cm):
    tps = np.diag(cm)
    fps = cm.sum(axis=0) - tps
    fns = cm.sum(axis=1) - tps
    iou = tps / (tps + fps + fns)
    # iou = np.nan_to_num(iou)
    iou = iou[~np.isnan(iou)]
    return iou.squeeze()

def cm_to_miou(cm):
    iou = cm_to_iou(cm)
    return iou.mean()

def read_metrics(metrics_path):
    with open(metrics_path, 'r') as f:
        lines = f.readlines()
        miou = float(lines[-1].split(':')[-1].strip())
    return miou

def add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou):
    metrics_dict['dataset'].append(dataset)
    metrics_dict['trajectory'].append(trajectory)
    metrics_dict['label_set'].append(label_set)
    metrics_dict['seg_method'].append(seg_method)
    metrics_dict['lulc_type'].append(lulc_type)
    metrics_dict['d3_type'].append(d3_type)
    metrics_dict['res'].append(res)
    metrics_dict['refine_type'].append(refine_type)
    metrics_dict['cm'].append(cm)
    # metrics_dict['iou'].append(iou)
    metrics_dict['miou'].append(miou)

In [3]:
common_idx2names = {
    0 : 'water',
    1 : 'trees',
    2 : 'low_vegetation',
    3 : 'built',
    4 : 'ground',
    5 : 'sky',
}

more_common_idx2names = {
    0 : 'water',
    1 : 'vegetation',
    2 : 'built',
    3 : 'ground',
    4 : 'sky',
}

most_common_idx2names = {
    0 : 'water',
    1 : 'ground',
    2 : 'sky',
}

In [4]:
cm_paths = glob.glob('./outputs_v2/*/*/*/*/*/*/*/*/confusion_matrix.npy')

In [5]:
metrics_dict = {
    'dataset' : [],
    'trajectory' : [],
    'label_set' : [],
    'seg_method' : [],
    'lulc_type' : [],
    'd3_type' : [],
    'res' : [],
    'refine_type' : [],
    'cm' : [],
    # 'iou' : [],
    'miou' : [],
}

for cm_filepath in tqdm.tqdm(cm_paths):
    iou_path = cm_filepath.replace('confusion_matrix.npy', 'metrics.txt')
    miou = read_metrics(iou_path)
    label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory = cm_filepath.split(os.path.sep)[-9:-1]
    cm = np.load(cm_filepath)
    iou = cm_to_iou(cm)
    miou = iou.mean()
    add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou)
    
    # Colorado has no DSM. Use dem 1m instead.
    # Lets us compare dem vs dsm
    if dataset == '2022-05-15_ColoradoRiver' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dsm', res, refine_type, dataset, trajectory, cm, miou)

    # Lets us compare dem vs dem_1m
    if dataset == '2023-03-XX_Duck' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dem_1m', res, refine_type, dataset, trajectory, cm, miou)

    


  0%|          | 0/620 [00:00<?, ?it/s]/tmp/ipykernel_1596798/257437282.py:5: RuntimeWarning: invalid value encountered in divide
  iou = tps / (tps + fps + fns)
100%|██████████| 620/620 [00:00<00:00, 6071.35it/s]


In [6]:
df = pd.DataFrame.from_dict(metrics_dict)

In [7]:
unique_ids = ['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']
grouped_df = df.groupby(unique_ids)

In [8]:
counts = grouped_df['miou'].transform('count')
cm_sum = grouped_df['cm'].transform('sum')
trajectory_avg_miou = grouped_df['miou'].transform('mean')

df['trajectory_count'] = counts
df['cm_sum'] = cm_sum
df['trajectory_avg_miou'] = trajectory_avg_miou
df['miou (total)'] = df.apply(lambda x: cm_to_miou(x['cm_sum']), axis=1)


/tmp/ipykernel_1596798/257437282.py:5: RuntimeWarning: invalid value encountered in divide
  iou = tps / (tps + fps + fns)


In [12]:
filter_cond = (
    (df['seg_method'] == 'open_sam_boxnms_0p50') & 
    (df['d3_type'] == 'dem') &
    (df['res'] == '1.0') &
    (df['lulc_type'] == 'dynamicworld') & 
    (df['refine_type'] == 'none') & 
    (df['label_set'] == 'most_common')
)

df[filter_cond][['dataset', 'trajectory', 'miou', 'miou (total)']]

,dataset,trajectory,miou,miou (total)
124,2023-03-XX_Duck,ONR_2023-03-21-09-59-39,0.715223,0.87645
126,2023-03-XX_Duck,ONR_2023-03-21-14-06-04,0.960929,0.87645
128,2023-03-XX_Duck,ONR_2023-03-21-19-55-11,0.908280,0.87645
130,2023-03-XX_Duck,ONR_2023-03-22-14-41-46,0.866337,0.87645
132,2022-05-15_ColoradoRiver,flight4,0.848590,0.87645
134,2022-05-15_ColoradoRiver,flight3,0.944690,0.87645
136,2022-05-15_ColoradoRiver,flight2,0.913996,0.87645
138,2021-09-09-KentuckyRiver,flight3-1,0.802536,0.87645
139,2021-09-09-KentuckyRiver,flight1-1,0.854024,0.87645
140,2021-09-09-KentuckyRiver,flight2-1,0.829736,0.87645


In [196]:
# df_agg = df.groupby(['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']).reset_index()
df_agg_ds_stats = df.drop(columns=['dataset', 'trajectory', 'cm', 'cm_sum', 'miou'])
df_agg_ds_stats

,label_set,seg_method,lulc_type,d3_type,res,refine_type,trajectory_count,trajectory_avg_miou,miou (total)
0,most_common,open_sam_boxnms_0p50,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.831408,0.835308
1,most_common,open_sam_boxnms_0p50,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.831408,0.835308
2,most_common,open_sam_boxnms_0p50,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.831408,0.835308
3,most_common,open_sam_boxnms_0p50,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.831408,0.835308
4,most_common,open_sam_boxnms_0p50,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.831408,0.835308
...,...,...,...,...,...,...,...,...,...
888,common,open_sam_boxnms_0p50,dynamicworld,dem_1m,1.0,none,11,0.441318,0.568481
889,common,open_sam_boxnms_0p50,dynamicworld,dem_1m,1.0,none,11,0.441318,0.568481
890,common,open_sam_boxnms_0p50,dynamicworld,dem_1m,1.0,none,11,0.441318,0.568481
891,common,open_sam_boxnms_0p50,dynamicworld,dem_1m,1.0,none,11,0.441318,0.568481


In [197]:
df_agg_ds_stats.drop_duplicates(inplace=True)

In [198]:
filter_cond = (
    (df_agg_ds_stats['seg_method'] == 'open_sam_boxnms_0p50') & 
    (df_agg_ds_stats['d3_type'] == 'dem') &
    (df_agg_ds_stats['res'] == '1.0') &
    (df_agg_ds_stats['lulc_type'] == 'dynamicworld') & 
    (df_agg_ds_stats['refine_type'] == 'none')
)

print(df_agg_ds_stats[filter_cond][['trajectory_avg_miou', 'miou (total)']].mean())
df_agg_ds_stats[filter_cond]


trajectory_avg_miou    0.609141
miou (total)           0.675154
dtype: float64


,label_set,seg_method,lulc_type,d3_type,res,refine_type,trajectory_count,trajectory_avg_miou,miou (total)
124,most_common,open_sam_boxnms_0p50,dynamicworld,dem,1.0,none,11,0.859671,0.876450
424,more_common,open_sam_boxnms_0p50,dynamicworld,dem,1.0,none,11,0.517656,0.571940
724,common,open_sam_boxnms_0p50,dynamicworld,dem,1.0,none,11,0.450098,0.577074


In [199]:
print(df_agg_ds_stats[filter_cond][['label_set', 'trajectory_avg_miou', 'miou (total)']].set_index('label_set').T[['common', 'more_common', 'most_common']].to_latex(float_format='{:.3f}'.format))

\begin{tabular}{lrrr}
\toprule
label_set & common & more_common & most_common \\
\midrule
trajectory_avg_miou & 0.450 & 0.518 & 0.860 \\
miou (total) & 0.577 & 0.572 & 0.876 \\
\bottomrule
\end{tabular}



In [200]:
df_agg_ds_stats['lulc_type'].unique()

array(['open_earth_map_unet_lc_naip_corrected',
       'open_earth_map_unet_lc_planet',
       'chesapeake_bay_swin_crossentropy_lc_naip_corrected',
       'chesapeake_bay_swin_crossentropy_lc_planet', 'dynamicworld'],
      dtype=object)

In [201]:
filter_cond = (
    (df_agg_ds_stats['seg_method'] == 'open_sam_boxnms_0p50') & 
    (df_agg_ds_stats['d3_type'] == 'dem') &
    # (df_agg_ds_stats['res'] == '1.0') &
    (df_agg_ds_stats['lulc_type'] == 'dynamicworld')
    # (df_agg_ds_stats['refine_type'] == 'none')
)

space_df = df_agg_ds_stats[filter_cond].copy()
# space_df.drop(columns=['seg_method', 'd3_type', 'lulc_type', 'miou (total)', 'trajectory_count'], inplace=True)
space_df['label_set'].str.strip()
space_df['res'] = space_df.apply(lambda x: float(x['res']), axis=1)
space_df.sort_values(by=['res'], inplace=True)
space_df['res'] = space_df.apply(lambda x: '{}m'.format(int(x['res'])), axis=1)

# pivoted_df = space_df.pivot(index=['res', 'refine_type'], columns='label_set', values='trajectory_avg_miou').reset_index()

In [202]:
import plotly.express as px

fig = px.line(space_df, x='res', y='trajectory_avg_miou',
              color='refine_type', facet_col='label_set', facet_col_spacing=0.075, markers=True, title=None,
              color_discrete_map={
                    'none': '#2E5C80',
                    'crf_planet': '#0091FF',
                    'crf_naip_naip-nir': '#FF7912'
              },
              category_orders={
                'label_set': ['common', 'more_common', 'most_common'],
                'res' : ['10m', '5m', '1m']},
              symbol='refine_type',
              symbol_map={
                    'none': 'x',
                    'crf_planet': 'x',
                    'crf_naip_naip-nir': 'x',
              },
              line_dash='refine_type',
              line_dash_map={
                        'none': 'dash',
                        'crf_planet': 'dot',
                        'crf_naip_naip-nir': 'solid'
                  },
              )
for anno in fig['layout']['annotations']:
    anno['text'] = ''
fig.update_yaxes(range=[0.36, 0.47], nticks=3, row=1, col=1)
fig.update_yaxes(range=[0.42, 0.54], nticks=3, row=1, col=2)
fig.update_yaxes(range=[0.79, 0.88], nticks=3, row=1, col=3)
# fig.update_xaxes(range=[11, -1])
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.1,
        xanchor="right",
        x=1,
        #hide
        
    ),
    showlegend=False,
    template='simple_white',
    width=600,
    height=200,
    margin=dict(l=30, r=20, t=0, b=0),
    font=dict(size=16)
)
fig.update_yaxes(matches=None, showticklabels=True)
fig.update_yaxes(title_text=None, row=1, col=1)
fig.update_xaxes(title_text=None)
fig.update_layout(showlegend=False, 
                  xaxis_type='category')
fig.show()
os.makedirs('spatial', exist_ok=True)
fig.write_image('spatial/spatial-plot-nolegend.pdf')
fig.write_image('spatial/spatial-plot-nolegend.pdf')